# Boto3 S3 Client Examples

## Install Requirements

In [ ]:
import sys
!pip install boto3

## Boto 3 Client Examples

The boto3 client is a more low level implementation that closely mirrors the API.


### Create the S3 Client

To connect to Ceph or other S3 API compatible storage, you'll need to specify `endpoint_url` but for AWS, you can omit that parameter

```python
s3 = boto3.client('s3',
                  endpoint_url='https://s3.amazonaws.com',
                  aws_access_key_id='your_aws_access_key_id',
                  aws_secret_access_key='your_aws_secret_access_key',
                  region_name='us-east-1')

```

In [ ]:
import os
import boto3

key_id = os.environ.get('S3_ACCESS_KEY_ID')
secret_key = os.environ.get('S3_SECRET_ACCESS_KEY')

session = boto3.session.Session(aws_access_key_id=key_id,
                                aws_secret_access_key=secret_key)

s3_client = boto3.client('s3',
                  aws_access_key_id=key_id,
                  aws_secret_access_key=secret_key)



### List Buckets

In [ ]:
response = s3_client.list_buckets()

for bucket in response['Buckets']:
    print(bucket["Name"])
    

### List objects in a single bucket

In [ ]:
paginator = s3_client.get_paginator('list_objects')
operation_parameters = {'Bucket': 'rhods-pilot'}
page_iterator = paginator.paginate(**operation_parameters)
for page in page_iterator:
    for obj in page['Contents']:
        print(obj['Key'], obj['LastModified'])  

### List objects in a "subdirectory" prefix

In [ ]:
paginator = s3_client.get_paginator('list_objects')
operation_parameters = {'Bucket': 'rhods-pilot',
                        'Prefix': 's3-folder'}
page_iterator = paginator.paginate(**operation_parameters)
for page in page_iterator:
    for obj in page['Contents']:
        print(obj['Key'], obj['LastModified'])  

### Download single file

In [ ]:
import os

s3_source_dir = ''
local_dest_dir = os.path.join(os.getcwd(), 'downloaded-bucket')

paginator = s3_client.get_paginator('list_objects')
bucket_name = 'rhods-pilot'
operation_parameters = {'Bucket': bucket_name}

page_iterator = paginator.paginate(**operation_parameters)
for page in page_iterator:
    for obj in page['Contents']:
        new_file_path = os.path.join(local_dest_dir, obj['Key'])
        print(new_file_path)

        try:
            os.makedirs(os.path.dirname(new_file_path))
        except OSError:
            pass

        if new_file_path.endswith('/'):
            continue

        if os.path.exists(new_file_path):
            os.remove(new_file_path)

        s3_client.download_file(bucket_name, obj['Key'], new_file_path)

### Download all files in a bucket

In [ ]:
import os

s3_source_dir = ''
local_dest_dir = os.path.join(os.getcwd(), 'downloaded-bucket')

paginator = s3_client.get_paginator('list_objects')
bucket_name = 'rhods-pilot'
operation_parameters = {'Bucket': bucket_name}

page_iterator = paginator.paginate(**operation_parameters)
for page in page_iterator:
    for obj in page['Contents']:
        new_file_path = os.path.join(local_dest_dir, obj['Key'])
        print(new_file_path)

        try:
            os.makedirs(os.path.dirname(new_file_path))
        except OSError:
            pass

        if new_file_path.endswith('/'):
            continue

        if os.path.exists(new_file_path):
            os.remove(new_file_path)

        s3_client.download_file(bucket_name, obj['Key'], new_file_path)


### Downloaded prefixed files in a "subdirectory"

In [ ]:
import os


s3_source_dir = 's3-folder'
local_dest_dir = os.path.join(os.getcwd(), 'downloaded-folder')
   
paginator = s3_client.get_paginator('list_objects')
bucket_name = 'rhods-pilot'
operation_parameters = {'Bucket': bucket_name,
                        'Prefix': s3_source_dir}
page_iterator = paginator.paginate(**operation_parameters)

for page in page_iterator:
    for obj in page['Contents']:
        stripped_key = obj['Key'].replace(s3_source_dir, '')
        if stripped_key.startswith('/'):
            stripped_key = stripped_key[1:]
        new_file_path = os.path.join(local_dest_dir, stripped_key)    
        print(new_file_path)

        try:
            os.makedirs(os.path.dirname(new_file_path))
        except OSError:
            pass

        if new_file_path.endswith('/'):
            continue

        if os.path.exists(new_file_path):
            os.remove(new_file_path)

        s3_client.download_file(bucket_name, obj['Key'], new_file_path)
